In [1]:
# Imports
import re
import numpy as np
import pandas as pd
import pickle
from datetime import datetime, timezone
from config import *

pd.options.mode.chained_assignment = None

In [2]:
try:
    save_path = save_path
except:
    save_path = 'data/minset.pkl.gz'

In [3]:
min2023 = pd.read_pickle('data/minbesluit.pkl.gz', compression='gzip')

In [4]:
# datums als '2 1 juni 2023' omzetten naar '21 juni 2023' en double whitespace weghalen
def join_numbers(s):
    s = ' '.join(s.split())
    return re.sub(r"(\s?\d)(\s)(\d\s)", '\g<1>\g<3>', s)

# elke hele zin pakken waarin 'deelbesluit' wordt genoemd
deelbesluit_regex1 = r"[A-Za-z\s0-9-]+deelbesluit[A-Za-z\s0-9-]+"

def regex_context_deelbesluit(s):
    if type(s) == float:
        return None
    else:
        search_result = re.findall(deelbesluit_regex1, join_numbers(s))
        if search_result != None:
            result = search_result
            if len(result) == 0:
                return None
            return result
        else:
            return None
        
# elke zin pakken waar 'verdaag' in wordt genoemd
verdaagd_regex1 = r"[A-Za-z\s0-9-]+verdaag[A-Za-z\s0-9-]+"

def regex_context_verdaagd(s):
    if type(s) == float:
        return None
    else:
        search_result = re.findall(verdaagd_regex1, join_numbers(s))
        if search_result != None:
            result = search_result
            if len(result) == 0:
                return None
            return result
        else:
            return search_result
        
def verschil(row):
    dagtekening = row['dagtekening_complete']
    verzoek = row['verzoek_datum']
    if isinstance(dagtekening, datetime) and isinstance(verzoek, datetime):
        verschil = dagtekening - verzoek
        if verschil.days < 0:
            return None
        else: 
            return verschil
    else:
        return None
    
def td_to_days(td):
    if td != None:
        return td.days

In [5]:
# deelbesluit context ophalen
min2023['deelbesluit_context'] = min2023['foi_bodyTextOCR'].apply(lambda x: regex_context_deelbesluit(x))

# opgehaalde contexten naar alle rijen van een document uitsmeren
min2023['deelbesluit_context'] = min2023['deelbesluit_context'].groupby(level=0).fillna(method='ffill')
min2023['deelbesluit_context'] = min2023['deelbesluit_context'].groupby(level=0).fillna(method='bfill')

In [6]:
# verdaagd context ophalen
min2023['verdaagd_context'] = min2023['foi_bodyTextOCR'].apply(lambda x: regex_context_verdaagd(x))

# opgehaalde contexten naar alle rijen van een document uitsmeren
min2023['verdaagd_context'] = min2023['verdaagd_context'].groupby(level=0).fillna(method='ffill')
min2023['verdaagd_context'] = min2023['verdaagd_context'].groupby(level=0).fillna(method='bfill')

In [7]:
min2023['looptijd'] = None
min2023['looptijd'] = min2023.apply(lambda x: td_to_days(verschil(x)), axis=1)
min2023['looptijd'] = min2023['looptijd'].groupby(level=0).fillna(method='ffill')
min2023['looptijd'] = min2023['looptijd'].groupby(level=0).fillna(method='bfill')

In [ ]:
min2023.to_pickle(save_path, compression='gzip')
print(f"Saved in: {save_path}")